- flip Y coordinate and attach additional 2017 data for data augmentation

In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold,KFold,GroupKFold
import xgboost as xgb
from datetime import datetime
from kaggle.competitions import nflrush
import math
import tqdm
from scipy.special import expit
from scipy.spatial import Delaunay, delaunay_plot_2d, Voronoi, voronoi_plot_2d, ConvexHull
pd.set_option("display.max_rows",1000)
env = nflrush.make_env()

In [2]:
def crps_score(y_pred, data):
    y_pred = y_pred.reshape(-1,199)
    y_true = np.array(data.get_label()).astype("int")
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return "CRPS", ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0])

def simple_crps(y_true, y_pred):
    y_true_cdf = np.zeros([y_true.shape[0], 199])
    for i, y in enumerate(y_true):
        y_true_cdf[i, y] = 1
    y_true_cdf = np.clip(np.cumsum(y_true_cdf, axis=1), 0, 1)
    y_pred = np.clip(np.cumsum(y_pred, axis=1), 0, 1)
    return ((y_true_cdf - y_pred) ** 2).sum(axis=1).sum(axis=0) / (199 * y_true.shape[0])

In [3]:
train = pd.read_csv('../input/nfl-big-data-bowl-2020/train.csv',low_memory=False)

# preprocess and feature engineering 

In [4]:
def transform_time_quarter(str1):
    return int(str1[:2])*60 + int(str1[3:5])
  
def transform_time_all(str1,quarter):
    if quarter<=4:
        return 15*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
    if quarter ==5:
        return 10*60 - (int(str1[:2])*60 + int(str1[3:5])) + (quarter-1)*15*60
      
def transform_height(te):
    return (int(te.split('-')[0])*12 + int(te.split('-')[1]))*2.54/100

def voronoi_volumes(points, selected_index):
    v = Voronoi(points)
    vol = np.zeros(v.npoints)
      
    for i, reg_num in enumerate(v.point_region):
        if reg_num == v.point_region[selected_index]:
            indices = v.regions[reg_num]
            if -1 in indices: # some regions can be opened
                vol = -999 ## insert missing value when the area is open
            else:
                vol = ConvexHull(v.vertices[indices]).volume      
            break
    return vol

def radius_calc(dist_to_ball):
    return 4 + 6 * (dist_to_ball >= 15) + (dist_to_ball ** 3) / 560 * (dist_to_ball < 15)

def compute_influence_to_ball(row):
    point = np.array([row["RushX"], row["RushY"]])
    theta = math.radians(row['Orientation'])
    speed = row['S']
    player_coords = row[['X', 'Y']].values
    dist_to_ball = row["DisToRusher"]    

    S_ratio = (speed / 13) ** 2    # we set max_speed to 13 m/s ((dominance_df["S"]*0.9144).max() #8.64m/s)
    RADIUS = radius_calc(dist_to_ball)  # updated

    S_matrix = np.matrix([[RADIUS * (1 + S_ratio), 0], [0, RADIUS * (1 - S_ratio)]])
    R_matrix = np.matrix([[np.cos(theta), - np.sin(theta)], [np.sin(theta), np.cos(theta)]])
    COV_matrix = np.dot(np.dot(np.dot(R_matrix, S_matrix), S_matrix), np.linalg.inv(R_matrix))
    
    norm_fact = (1 / 2 * np.pi) * (1 / np.sqrt(np.linalg.det(COV_matrix)))    
    mu_play = player_coords + speed * np.array([np.cos(theta), np.sin(theta)]) / 2
    
    intermed_scalar_player = np.dot(np.dot((player_coords - mu_play),
                                    np.linalg.inv(COV_matrix)),
                             np.transpose((player_coords - mu_play)))
    player_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_player[0, 0])
    
    intermed_scalar_point = np.dot(np.dot((point - mu_play), 
                                    np.linalg.inv(COV_matrix)), 
                             np.transpose((point - mu_play)))
    point_influence = norm_fact * np.exp(- 0.5 * intermed_scalar_point[0, 0])
    
    return point_influence / player_influence

In [5]:
remove_features = ['GameId','PlayId','DisplayName','GameClock','TimeHandoff','TimeSnap', 'PlayDirection', 'TeamOnOffense', 
                    'PlayerBirthDate', 'is_run', 'NflIdRusher', 'date_game', 'RushX', 'RushY', 'PossessionTeam', 
                   'FieldPosition', 'Position', 'PlayerHeight', 'HomeTeamAbbr', 'VisitorTeamAbbr', 'Turf', 'Quarter', 'OffenseFormation'
                  , 'RushX_0.5s', 'RushY_0.5s', 'RushX_1s', 'RushY_1s',  'Team', 'WindSpeed',
                   'GameWeather', 'WindDirection', 'Location', 'Stadium', 'StadiumType', 'Temperature', 'DefensePersonnel'
                  , 'PlayerCollegeName', 'OffensePersonnel'] #"Humidity"

In [6]:
def transform_data(df, y_flip):
    df.loc[df.VisitorTeamAbbr == "ARI",'VisitorTeamAbbr'] = "ARZ"
    df.loc[df.HomeTeamAbbr == "ARI",'HomeTeamAbbr'] = "ARZ"

    df.loc[df.VisitorTeamAbbr == "BAL",'VisitorTeamAbbr'] = "BLT"
    df.loc[df.HomeTeamAbbr == "BAL",'HomeTeamAbbr'] = "BLT"

    df.loc[df.VisitorTeamAbbr == "CLE",'VisitorTeamAbbr'] = "CLV"
    df.loc[df.HomeTeamAbbr == "CLE",'HomeTeamAbbr'] = "CLV"

    df.loc[df.VisitorTeamAbbr == "HOU",'VisitorTeamAbbr'] = "HST"
    df.loc[df.HomeTeamAbbr == "HOU",'HomeTeamAbbr'] = "HST"

    df['is_run'] = df.NflId == df.NflIdRusher

    df['S'] = 10 * df['Dis']
    df.loc[df.Season == 2017, 'A'] = df[df.Season == 2018]['A'].mean()
    
    df['ToLeft'] = df.PlayDirection == "left"
    df['TeamOnOffense'] = "home"
    df.loc[df.PossessionTeam != df.HomeTeamAbbr, 'TeamOnOffense'] = "away"
    df['OnOffense'] = df.Team == df.TeamOnOffense 
    df['YardLine_std'] = 100 - df.YardLine.copy()
    df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
            'YardLine_std'
             ] = df.loc[df.FieldPosition.fillna('') == df.PossessionTeam,  
              'YardLine']
    df['X_std'] = df.X.copy()
    df.loc[df.ToLeft, 'X_std'] = 120 - df.loc[df.ToLeft, 'X'] 
    df['Y_std'] = df.Y.copy()
    df.loc[df.ToLeft, 'Y_std'] = 53.3 - df.loc[df.ToLeft, 'Y'] 
    df['Orientation_std'] = df.Orientation.copy()
    df.loc[df.ToLeft, 'Orientation_std'] = np.mod(180 + df.loc[df.ToLeft, 'Orientation_std'], 360)
    df['Dir_std'] = df.Dir.copy()
    df.loc[df.ToLeft, 'Dir_std'] = np.mod(180 + df.loc[df.ToLeft, 'Dir_std'], 360)
    df.loc[df['Season'] == 2017, 'Orientation_std'] = np.mod(90 + df.loc[df['Season'] == 2017, 'Orientation_std'], 360) 
    df.drop(["X", "Y", "Orientation", "YardLine", "Dir", "ToLeft"], axis=1, inplace=True)
    df.rename(columns={'X_std': 'X', 'Y_std': 'Y', 'Orientation_std': 'Orientation', 'Dir_std': 'Dir', "YardLine_std": "YardLine"}, inplace=True)
    
    if y_flip: # flip Y for data augmentation
        df["Y"] = 53.3 - df['Y'] 
    
    df['date_game'] = df.GameId.map(lambda x:pd.to_datetime(str(x)[:8]))
    df['age'] = (df.date_game.map(pd.to_datetime) - df.PlayerBirthDate.map(pd.to_datetime)).map(lambda x:x.days)/365

    df["Momentum"] = df["S"] * df["PlayerWeight"]
    
    df["Dir_cos"] = df["Dir"].apply(lambda x : np.cos((450-x) * np.pi/ 180))
    
    df["Momentum_cos"] = df["Momentum"] * df["Dir_cos"]
    
    df["Dir_sin"] = df["Dir"].apply(lambda x : np.sin((450-x) * np.pi/ 180))
    
    df["S_cos"] = df["S"] * df["Dir_cos"]
    df["S_sin"] = df["S"] * df["Dir_sin"]
    df["X_0.5s"] = df["X"] + df["S_cos"] * 0.5
    df["Y_0.5s"] = df["Y"] + df["S_sin"] * 0.5
    df["X_1s"] = df["X"] + df["S_cos"] * 1
    df["Y_1s"] = df["Y"] + df["S_sin"] * 1

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X"].agg(np.mean)[1::2])
    df["RushX"] = np.repeat(rusher_x, 22) # repeat each elemnt 22 times
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y"].agg(np.mean)[1::2])
    df["RushY"] = np.repeat(rusher_y, 22) 
    df["DisToRusher"] = np.sqrt((df["X"] - df["RushX"]) ** 2 + (df["Y"] - df["RushY"]) ** 2)
    df["TackleTimeToRusher"] = df["DisToRusher"] / df["S"] # includes nan when the speed of rusher is 0
    
    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X_0.5s"].agg(np.mean)[1::2])
    df["RushX_0.5s"] = np.repeat(rusher_x, 22) 
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y_0.5s"].agg(np.mean)[1::2])
    df["RushY_0.5s"] = np.repeat(rusher_y, 22) 
    df["DisToRusher_0.5s"] = np.sqrt((df["X_0.5s"] - df["RushX_0.5s"]) ** 2 + (df["Y_0.5s"] - df["RushY_0.5s"]) ** 2)
    df["TackleTimeToRusher_0.5s"] = df["DisToRusher_0.5s"] / df["S"]

    rusher_x = np.array(df.groupby(["PlayId", "is_run"])["X_1s"].agg(np.mean)[1::2])
    df["RushX_1s"] = np.repeat(rusher_x, 22) 
    rusher_y = np.array(df.groupby(["PlayId", "is_run"])["Y_1s"].agg(np.mean)[1::2])
    df["RushY_1s"] = np.repeat(rusher_y, 22) 
    df["DisToRusher_1s"] = np.sqrt((df["X_1s"] - df["RushX_1s"]) ** 2 + (df["Y_1s"] - df["RushY_1s"]) ** 2)
    df["TackleTimeToRusher_1s"] = df["DisToRusher_1s"] / df["S"]
    
    #df["X_0.5s"] = df["X"] + (df["S_cos"] * 0.5 + 0.5 * df["A"] * df["Dir_cos"] * (0.5**2) )
    #df["Y_0.5s"] = df["Y"] + (df["S_sin"] * 0.5 + 0.5 * df["A"] * df["Dir_sin"] * (0.5**2) )
    #df["X_1s"] = df["X"] + (df["S_cos"] * 1 + 0.5 * df["A"] * df["Dir_cos"] * (1**2) ) 
    #df["Y_1s"] = df["Y"] + (df["S_sin"] * 1 + 0.5 * df["A"] * df["Dir_sin"] * (1**2) ) 
    #df["X_2s"] = df["X"] + (df["S_cos"] * 2 + 0.5 * df["A"] * df["Dir_cos"] * (2**2) ) 
    #df["Y_2s"] = df["Y"] + (df["S_sin"] * 2 + 0.5 * df["A"] * df["Dir_sin"] * (2**2) )
    
    df["AttackAngle"] =  np.arctan((df["Y"] - df["RushY"]) / (df["X"] - df["RushX"])) * 180 / np.pi
    
    df_single = df[df.is_run==True].copy()
        
    df_single['time_end'] = df_single.apply(lambda x:transform_time_all(x.loc['GameClock'],x.loc['Quarter']),axis=1)
                                                                                                                
    df_single['DefendersInTheBox_vs_Distance'] = df_single['DefendersInTheBox'] / df_single['Distance']
                                                                   
    df_single["DisToQB"] = np.array(df[(df.Position=="QB") | (df.Position=="C")].groupby(["PlayId"]).agg(np.mean)["DisToRusher"])

    df_single['runner_height'] = df_single.PlayerHeight.map(transform_height)
    df_single.drop(remove_features,axis=1,inplace=True) 

    tmp = df.groupby(["PlayId", "OnOffense"]).agg(np.mean)[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseAveX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseAveX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseAveY"] = np.array(tmp[0::2]["Y"]) 
    df_single["OffenseAveY"] = np.array(tmp[1::2]["Y"]) 
    df_single["DefenseAveMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseAveMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseAveMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseAveMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])
    
    #tmp = df.groupby(["PlayId", "OnOffense"]).agg(np.mean)[["X_0.5s", "Y_0.5s", "X_1s", "Y_1s"]]
    #df_single["DefenseAveX_0.5s"] = np.array(tmp[0::2]["X_0.5s"])
    #df_single["OffenseAveX_0.5s"] = np.array(tmp[1::2]["X_0.5s"])
    #df_single["DefenseAveY_0.5s"] = np.array(tmp[0::2]["Y_0.5s"]) 
    #df_single["OffenseAveY_0.5s"] = np.array(tmp[1::2]["Y_0.5s"]) 
    #df_single["DefenseAveX_1s"] = np.array(tmp[0::2]["X_1s"])
    #df_single["OffenseAveX_1s"] = np.array(tmp[1::2]["X_1s"])
    #df_single["DefenseAveY_1s"] = np.array(tmp[0::2]["Y_1s"]) 
    #df_single["OffenseAveY_1s"] = np.array(tmp[1::2]["Y_1s"])
    
    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X", "Y", "Momentum", "Momentum_cos"]]
    df_single["DefenseStdX"] = np.array(tmp[0::2]["X"])
    df_single["OffenseStdX"] = np.array(tmp[1::2]["X"])
    df_single["DefenseStdY"] = np.array(tmp[0::2]["Y"])
    df_single["OffenseStdY"] = np.array(tmp[1::2]["Y"])
    df_single["DefenseStdMomentum"] = np.array(tmp[0::2]["Momentum"])
    df_single["OffenseStdMomentum"] = np.array(tmp[1::2]["Momentum"])
    df_single["DefenseStdMomentum_cos"] = np.array(tmp[0::2]["Momentum_cos"])
    df_single["OffenseStdMomentum_cos"] = np.array(tmp[1::2]["Momentum_cos"])
    
    tmp = df.groupby(["PlayId", "OnOffense"]).agg(["std"])[["X_0.5s", "Y_0.5s", "X_1s", "Y_1s"]]
    df_single["DefenseStdX_0.5s"] = np.array(tmp[0::2]["X_0.5s"])
    df_single["OffenseStdX_0.5s"] = np.array(tmp[1::2]["X_0.5s"])
    df_single["DefenseStdY_0.5s"] = np.array(tmp[0::2]["Y_0.5s"]) 
    df_single["OffenseStdY_0.5s"] = np.array(tmp[1::2]["Y_0.5s"]) 
    df_single["DefenseStdX_1s"] = np.array(tmp[0::2]["X_1s"])
    df_single["OffenseStdX_1s"] = np.array(tmp[1::2]["X_1s"])
    df_single["DefenseStdY_1s"] = np.array(tmp[0::2]["Y_1s"]) 
    df_single["OffenseStdY_1s"] = np.array(tmp[1::2]["Y_1s"])
    
    df_single["RunnerToDefenseCentoid"] = np.sqrt((df_single["X"] - df_single["DefenseAveX"]) ** 2 + (df_single["Y"] - df_single["DefenseAveY"]) ** 2)
    df_single["RunnerToOffenseCentoid"] = np.sqrt((df_single["X"] - df_single["OffenseAveX"]) ** 2 + (df_single["Y"] - df_single["OffenseAveY"]) ** 2)

    tmp_max = df.groupby(["PlayId", "OnOffense"])["X"].max()
    tmp_min = df.groupby(["PlayId", "OnOffense"])["X"].min()
    df_single["DefenseSpreadX"] = np.array(tmp_max[0::2]- tmp_min[0::2])
    df_single["OffenseSpreadX"] = np.array(tmp_max[1::2]- tmp_min[1::2])

    df_single["RunnerToScrimmage"] = df_single["X"] - (df_single["YardLine"] + 10)

    df_single["MinTackleTime"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher"].min()[0::2])
    df_single["MinTackleTime_0.5s"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher_0.5s"].min()[0::2])
    df_single["MinTackleTime_1s"] = np.array(df.groupby(["PlayId", "OnOffense"])["TackleTimeToRusher_1s"].min()[0::2])
    df_single["1stDefender_Momentum_cos"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["Momentum_cos"])
    first_offenser_index =df.groupby(["PlayId", "OnOffense"])["DisToRusher"].nsmallest(2)[3::4].reset_index()["level_2"]
    df_single["1stOffenser_Momentum_cos"] = np.array(df.loc[first_offenser_index]["Momentum_cos"])
    df_single["1stDefender_AttackAngle"] = np.array(df.loc[df.groupby(["PlayId", "OnOffense"])["DisToRusher"].idxmin()[0::2]]["AttackAngle"])

    df_single.fillna(-999,inplace=True) 
    remove_features2 = ["OnOffense", "DisToRusher", "TackleTimeToRusher", "OffenseAveX", "OffenseAveY", "AttackAngle", "Dis",
                       "DisToRusher_0.5s", "TackleTimeToRusher_0.5s", "DisToRusher_1s", "TackleTimeToRusher_1s"]
    df_single.drop(remove_features2, axis=1, inplace=True)

    return df_single

In [7]:
%%time
train_single = transform_data(train, y_flip = False).reset_index(drop=True)
train_single_flip = transform_data(train, y_flip = True).reset_index(drop=True)

CPU times: user 15min 22s, sys: 7.91 s, total: 15min 30s
Wall time: 15min 22s


In [8]:
new_train = pd.concat([train_single, train_single_flip[train_single_flip.Season == 2017]], sort=False).reset_index(drop=True)
y_train = new_train.Yards + 99 # to categorize
X_train = new_train.drop(['Yards'],axis=1)
for f in X_train.columns:
    if X_train[f].dtype=='object': 
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(X_train[f])+[-999])
        X_train[f] = lbl.transform(list(X_train[f]))

# modelling

In [9]:
# modified CV
df_20172018 = pd.concat([X_train, y_train], axis=1)
df_2017 = df_20172018[df_20172018.Season==2017].copy()
df_2018 = df_20172018[df_20172018.Season==2018].copy()
X_2017 = df_2017.drop(['Yards'],axis=1)
y_2017 = df_2017.Yards
X_2018 = df_2018.drop(['Yards'],axis=1)
y_2018 = df_2018.Yards
groups = np.array(X_2018.Week)
n_folds= 3
gkf=GroupKFold(n_splits = n_folds)
average_crps = 0
models = []
crps_scores = []
xgb_params = {
    "objective" : "multi:softprob",
    "eval_metric" : "mlogloss", 
    "max_depth" : 4,
    "boosting": 'gbdt',
    "num_class": 199,
    "num_leaves" : 13,
    "learning_rate" : 0.05,
}
evals_result = {}
num_boost_round=100000
# 2017 data: training only, 2018 data: separate by cross validation
for i , (train_index, test_index) in enumerate(gkf.split(X_2018, y_2018, groups)):
    X_train_2018 = X_2018.iloc[train_index,:]
    y_train_2018 = y_2018.iloc[train_index]
    X_test2 = X_2018.iloc[test_index,:]
    y_test2 = y_2018.iloc[test_index]
    X_train2 = pd.concat([X_2017, X_train_2018])
    y_train2 = pd.concat([y_2017, y_train_2018])
    xgb_train = xgb.DMatrix(X_train2, label = y_train2)
    xgb_eval = xgb.DMatrix(X_test2, label = y_test2)
    watchlist = [(xgb_eval, "eval")]
    
    clf = xgb.train(
        xgb_params, xgb_train, num_boost_round, watchlist,
        early_stopping_rounds=10,
        evals_result=evals_result,
        feval=crps_score,
        verbose_eval = 100
    )
    
    models.append(clf)
    temp_predict = clf.predict(xgb_eval, ntree_limit=clf.best_ntree_limit)
    score = simple_crps(y_test2, temp_predict)
    print(score)
    crps_scores.append(score)
    average_crps += score / n_folds
    if i == 0:
        feature_importance_df = pd.DataFrame(clf.get_score(importance_type="total_gain").items(), columns =["Features", "Fold_"+str(i+1)])
    else:
        feature_importance_df["Fold_"+str(i+1)] = list(clf.get_score(importance_type="total_gain").values())
    
feature_importance_df["Average"] = np.mean(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Std"] = np.std(feature_importance_df.iloc[:,1:n_folds+1], axis=1)
feature_importance_df["Cv"] = feature_importance_df["Std"] / feature_importance_df["Average"]
print('crps list:', crps_scores)
print('average crps:',average_crps)

/opt/conda/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0]	eval-mlogloss:4.89419	eval-CRPS:0.080578
Multiple eval metrics have been passed: 'eval-CRPS' will be used for early stopping.

Will train until eval-CRPS hasn't improved in 10 rounds.
[100]	eval-mlogloss:2.86919	eval-CRPS:0.014065
Stopping. Best iteration:
[184]	eval-mlogloss:2.8378	eval-CRPS:0.013652

0.013652324104453679
[0]	eval-mlogloss:4.89059	eval-CRPS:0.08064
Multiple eval metrics have been passed: 'eval-CRPS' will be used for early stopping.

Will train until eval-CRPS hasn't improved in 10 rounds.
[100]	eval-mlogloss:2.84042	eval-CRPS:0.013991
[200]	eval-mlogloss:2.81233	eval-CRPS:0.013564
Stopping. Best iteration:
[195]	eval-mlogloss:2.81129	eval-CRPS:0.013563

0.013563131781292509
[0]	eval-mlogloss:4.8876	eval-CRPS:0.080598
Multiple eval metrics have been passed: 'eval-CRPS' will be used for early stopping.

Will train until eval-CRPS hasn't improved in 10 rounds.
[100]	eval-mlogloss:2.827	eval-CRPS:0.014005
[200]	eval-mlogloss:2.80072	eval-CRPS:0.013579
Stopping. Best i

In [10]:
X_train.columns

Index(['S', 'A', 'NflId', 'JerseyNumber', 'Season', 'Down', 'Distance',
       'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'DefendersInTheBox',
       'PlayerWeight', 'Week', 'Humidity', 'YardLine', 'X', 'Y', 'Orientation',
       'Dir', 'age', 'Momentum', 'Dir_cos', 'Momentum_cos', 'Dir_sin', 'S_cos',
       'S_sin', 'X_0.5s', 'Y_0.5s', 'X_1s', 'Y_1s', 'time_end',
       'DefendersInTheBox_vs_Distance', 'DisToQB', 'runner_height',
       'DefenseAveX', 'DefenseAveY', 'DefenseAveMomentum',
       'OffenseAveMomentum', 'DefenseAveMomentum_cos',
       'OffenseAveMomentum_cos', 'DefenseStdX', 'OffenseStdX', 'DefenseStdY',
       'OffenseStdY', 'DefenseStdMomentum', 'OffenseStdMomentum',
       'DefenseStdMomentum_cos', 'OffenseStdMomentum_cos', 'DefenseStdX_0.5s',
       'OffenseStdX_0.5s', 'DefenseStdY_0.5s', 'OffenseStdY_0.5s',
       'DefenseStdX_1s', 'OffenseStdX_1s', 'DefenseStdY_1s', 'OffenseStdY_1s',
       'RunnerToDefenseCentoid', 'RunnerToOffenseCentoid', 'DefenseSpreadX',

In [11]:
feature_importance_df.sort_values("Average").head(80).reset_index(drop=True)

,Features,Fold_1,Fold_2,Fold_3,Average,Std,Cv
0,Season,306.152989,1416.834290,474.851772,732.613017,488.694781,0.667057
1,DefenseStdY_0.5s,1618.702439,1317.404208,1473.307590,1469.804746,123.029423,0.083705
2,DefenseStdY_1s,1370.518016,1629.916508,1434.485131,1478.306552,110.339271,0.074639
3,Orientation,1837.880744,1503.074844,1285.872233,1542.275940,227.054903,0.147221
4,Y_1s,1270.604394,3183.521858,1332.391172,1928.839141,887.553169,0.460149
5,Down,1214.070374,3317.418770,1984.210477,2171.899874,868.883997,0.400057
6,X_0.5s,2348.121640,1460.939630,3187.529714,2332.196995,704.967387,0.302276
7,OffenseStdX_0.5s,7378.476242,457.933577,3148.343453,3661.584424,2848.513052,0.777945
8,runner_height,3801.962099,3003.805166,5500.163548,4101.976937,1040.979657,0.253775
9,Distance,3404.681347,7255.731697,1762.537031,4140.983358,2302.231150,0.555962


# prediction

In [12]:
for (test_df, sample_prediction_df) in env.iter_test():
    X_test = transform_data(test_df, y_flip = False)
    for f in X_test.columns:
        if X_test[f].dtype=='object':
            X_test[f] = X_test[f].map(lambda x:x if x in set(X_train[f]) else -999)
    for f in X_test.columns:
        if X_test[f].dtype=='object': 
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(X_train[f])+[-999])
            X_test[f] = lbl.transform(list(X_test[f])) 
    pred_value = 0
    dtest = xgb.DMatrix(X_test)
    for model in models:
        pred_value += model.predict(dtest, ntree_limit = model.best_ntree_limit)[0] / n_folds
    pred_data = np.clip(np.cumsum(pred_value), 0, 1)
    pred_data = np.array(pred_data).reshape(1,199)
    pred_target = pd.DataFrame(index = sample_prediction_df.index, \
                               columns = sample_prediction_df.columns, \
                               data = pred_data)
    env.predict(pred_target)
env.write_submission_file()

Your submission file has been saved!  Once you `Commit` your Notebook and it finishes running, you can submit the file to the competition from the Notebook Viewer `Output` tab.
